# Day 22: Sporifica Virus

Diagnostics indicate that the local grid computing cluster has been contaminated with the Sporifica Virus. The grid computing cluster is a seemingly-infinite two-dimensional grid of compute nodes. Each node is either clean or infected by the virus.

To prevent overloading the nodes (which would render them useless to the virus) or detection by system administrators, exactly one virus carrier moves through the network, infecting or cleaning nodes as it moves. The virus carrier is always located on a single node in the network (the current node) and keeps track of the direction it is facing.

To avoid detection, the virus carrier works in bursts; in each burst, it wakes up, does some work, and goes back to sleep. The following steps are all executed in order one time each burst:

- If the current node is infected, it turns to its right. Otherwise, it turns to its left. (Turning is done in-place; the current node does not change.)
- If the current node is clean, it becomes infected. Otherwise, it becomes cleaned. (This is done after the node is considered for the purposes of changing direction.)
- The virus carrier moves forward one node in the direction it is facing.
- Diagnostics have also provided a map of the node infection status (your puzzle input). Clean nodes are shown as `.`; infected nodes are shown as `#`. This map only shows the center of the grid; there are many more nodes beyond those shown, but none of them are currently infected.

The virus carrier begins in the middle of the map facing up.

In [1]:
import numpy as np

CLOCKWISE = ['up', 'right', 'down', 'left']
DELTA = {'up': (-1, 0), 'down': (1, 0), 'left': (0, -1), 'right': (0, 1)}
TRANSITIONS = {
    '#': ('.', +1),
    '.': ('#', -1),
}


def parse(s):
    return np.array([list(row) for row in s.strip().splitlines()])


def pretty_print(info):
    out = [list(' %s ' % ' '.join(row)) for row in info['map']]
    pos = info['pos']
    out[pos[0]][2 * pos[1]] = '['
    out[pos[0]][2 * pos[1] + 2] = ']'
    print('\n'.join(''.join(row) for row in out))


def run(map, n):
    map = map.copy()

    # start in the center of the map, facing upwards
    assert map.shape[0] % 2 == 1 and map.shape[1] % 2 == 1
    pos = map.shape[0] // 2, map.shape[1] // 2
    dir = 'up'

    # run simulation
    infections = 0
    for _ in range(n):
        # determine new cell content and direction
        map[pos], rel_dir = TRANSITIONS[map[pos]]
        dir = CLOCKWISE[(CLOCKWISE.index(dir) + rel_dir) % 4]

        # count infection
        if map[pos] == '#':
            infections += 1

        # update position
        delta = DELTA[dir]
        pos = pos[0] + delta[0], pos[1] + delta[1]

        # grow map?
        if not (0 <= pos[0] < map.shape[0] and 0 <= pos[1] < map.shape[1]):
            new_map = np.full((map.shape[0] + 2, map.shape[1] + 2), '.')
            new_map[1:-1, 1:-1] = map
            map = new_map
            pos = pos[0] + 1, pos[1] + 1

    return {'map': map, 'pos': pos, 'dir': dir, 'infections': infections}

For example, suppose you are given a map like this:

In [2]:
example = parse("""
..#
#..
...
""")
pretty_print(run(example, 0))

 . . # 
 #[.]. 
 . . . 


The virus carrier is on a clean node, so it turns left, infects the node, and moves left:

In [3]:
pretty_print(run(example, 1))

 . . # 
[#]# . 
 . . . 


The virus carrier is on an infected node, so it turns right, cleans the node, and moves up:

In [4]:
pretty_print(run(example, 2))

[.]. # 
 . # . 
 . . . 


Four times in a row, the virus carrier finds a clean, infects it, turns left, and moves forward, ending in the same place and still facing up:

In [5]:
pretty_print(run(example, 2 + 4))

 . . . . . 
 #[#]. # . 
 # # # . . 
 . . . . . 
 . . . . . 


Now on the same node as before, it sees an infection, which causes it to turn right, clean the node, and move forward:

In [6]:
pretty_print(run(example, 2 + 4 + 1))

 . . . . . 
 # .[.]# . 
 # # # . . 
 . . . . . 
 . . . . . 


After the above actions, a total of 7 bursts of activity had taken place. Of them, 5 bursts of activity caused an infection.

In [7]:
assert run(example, 7)['infections'] == 5

After a total of 70, the grid looks like this, with the virus carrier facing up:

In [8]:
pretty_print(run(example, 70))

 . . . . . # # . . 
 . . . . # . . # . 
 . . . # . . . . # 
 . . # . #[.]. . # 
 . . # . # . . # . 
 . . . . . # # . . 
 . . . . . . . . . 
 . . . . . . . . . 
 . . . . . . . . . 


By this time, 41 bursts of activity caused an infection (though most of those nodes have since been cleaned).

In [9]:
assert run(example, 70)['infections'] == 41

After a total of 10000 bursts of activity, 5587 bursts will have caused an infection.

In [10]:
assert run(example, 10_000)['infections'] == 5587

Given your actual map, after 10000 bursts of activity, how many bursts cause a node to become infected? (Do not count nodes that begin infected.)

In [11]:
puzzle = parse(open('22.input').read())
run(puzzle, 10_000)['infections']

5404

# Part Two

As you go to remove the virus from the infected nodes, it evolves to resist your attempt.

Now, before it infects a clean node, it will weaken it to disable your defenses. If it encounters an infected node, it will instead flag the node to be cleaned in the future. So:

- Clean nodes become weakened.
- Weakened nodes become infected.
- Infected nodes become flagged.
- Flagged nodes become clean.
- Every node is always in exactly one of the above states.

The virus carrier still functions in a similar way, but now uses the following logic during its bursts of action:

- Decide which way to turn based on the current node:
  - If it is clean, it turns left.
  - If it is weakened, it does not turn, and will continue moving in the same direction.
  - If it is infected, it turns right.
  - If it is flagged, it reverses direction, and will go back the way it came.
- Modify the state of the current node, as described above.
- The virus carrier moves forward one node in the direction it is facing.

In [12]:
TRANSITIONS = {
    '.': ('W', -1),
    'W': ('#', 0),
    '#': ('F', +1),
    'F': ('.', 2),
}

{'#': ('F', 1), '.': ('W', -1), 'F': ('.', 2), 'W': ('#', 0)}

Start with the same map (still using `.` for clean and `#` for infected) and still with the virus carrier starting in the middle and facing up.

Using the same initial state as the previous example, and drawing weakened as `W` and flagged as `F`, the middle of the infinite grid looks like this, with the virus carrier's position again marked with `[ ]`:

In [13]:
pretty_print(run(example, 0))

 . . # 
 #[.]. 
 . . . 


This is the same as before, since no initial nodes are weakened or flagged. The virus carrier is on a clean node, so it still turns left, instead weakens the node, and moves left:

In [14]:
pretty_print(run(example, 1))

 . . # 
[#]W . 
 . . . 


The virus carrier is on an infected node, so it still turns right, instead flags the node, and moves up:

In [15]:
pretty_print(run(example, 2))

[.]. # 
 F W . 
 . . . 


This process repeats three more times, ending on the previously-flagged node and facing right:

In [16]:
pretty_print(run(example, 2 + 3))

 . . . . . 
 W W . # . 
 W[F]W . . 
 . . . . . 
 . . . . . 


Finding a flagged node, it reverses direction and cleans the node:

In [17]:
pretty_print(run(example, 2 + 3 + 1))

 . . . . . 
 W W . # . 
[W]. W . . 
 . . . . . 
 . . . . . 


The weakened node becomes infected, and it continues in the same direction:

In [18]:
pretty_print(run(example, 2 + 3 + 2))

 . . . . . . . 
 . . . . . . . 
 . W W . # . . 
[.]# . W . . . 
 . . . . . . . 
 . . . . . . . 
 . . . . . . . 


Of the first 100 bursts, 26 will result in infection.

In [19]:
assert run(example, 100)['infections'] == 26

Unfortunately, another feature of this evolved virus is speed; of the first 10_000_000 bursts, 2_511_944 will result in infection.

In [20]:
%%time
assert run(example, 10_000_000)['infections'] == 2_511_944

CPU times: user 20.9 s, sys: 162 ms, total: 21 s
Wall time: 21.3 s


Given your actual map, after 10_000_000 bursts of activity, how many bursts cause a node to become infected? (Do not count nodes that begin infected.)

In [21]:
%%time
run(puzzle, 10_000_000)['infections']

CPU times: user 20.3 s, sys: 114 ms, total: 20.5 s
Wall time: 20.7 s


2511672